In [161]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import imageio # pip install imageio
from scipy import ndimage, misc
import visvis as vv # pip install visvis

import sklearn


# Few of certain letters


In [174]:
letterDict={
  "a": 0,"b": 1,"c": 2,"d": 3,"e": 4,"f": 5,"g": 6,"h": 7,"i": 8,
    "j": 9,"k": 10,"l": 11,"m": 12,"n": 13,"o": 14,"p": 15,"q": 16,"r": 17,
    "s": 18,"t": 19,"u": 20,"v": 21,"w": 22,"x": 23,"y": 24,"z": 25
}

def readPicture(df_complete, directory,letter_value):
    im = imageio.imread(directory)
    im.shape
    
    print("Letter value: ", directory, end="\r")


    im=np.reshape(im,400)
    df = pd.DataFrame(data=im).T
    y = np.repeat(0,400)
    df['Letter'] = letter_value
    
    return df_complete.append(df)


In [175]:
import os
all_imgs = pd.DataFrame()
# imageio.imread(directory)

def read_all_images(upper_directory,all_imgs):
    number_of_images = 0
    for subdir, dirs, files in os.walk(upper_directory):
        for file in files:
            number_of_images+=1
            #print os.path.join(subdir, file)
            filepath = subdir + os.sep + file
            if filepath.endswith(".jpg"):
                # print(filepath)
                # Rotate every image that is being loaded:
            
                x = filepath.split("/")
                rotate_image(filepath,x[-1],20)
                rotate_image(filepath,x[-1],90)
                rotate_image(filepath,x[-1],320)
                rotate_image(filepath,x[-1],270)
                
                # The label for the data:
                letter = (x[-1][0])
                letter_val = letterDict.get(letter)
                all_imgs = readPicture(all_imgs,filepath,letter_val)
    print("image")
    # vv.imshow("./dataset3/chars74k-lite/augmented/rotated_neg_a102.jpg")
    print("number_of_images", number_of_images)
    return all_imgs

def rotate_image(image_path,ending,deg):
    out_path = "./dataset3/augmented/"
    image_to_rotate = plt.imread(image_path)
    
    # Rotating the image:
    rotated_positive = ndimage.rotate(image_to_rotate, deg, reshape=False)
   
    fullpath = os.path.join(out_path, 'rotated_'+ str(deg) + '_' + ending)
    imageio.imwrite(fullpath, rotated_positive)
               
# Unaugmented data, no feature selection:            
alldata = read_all_images("./dataset3/chars74k-lite",all_imgs)
alldata

imager value:  ./dataset3/chars74k-lite/z/z_9.jpggg ./dataset3/chars74k-lite/c/c_233.jpg./dataset3/chars74k-lite/e/e_333.jpg ./dataset3/chars74k-lite/n/n_303.jpg ./dataset3/chars74k-lite/p/p_161.jpg ./dataset3/chars74k-lite/r/r_245.jpg
number_of_images 7114


,0,1,2,3,4,5,6,7,8,9,...,391,392,393,394,395,396,397,398,399,Letter
0,193,175,177,191,177,188,194,177,197,157,...,175,191,175,183,188,225,185,167,193,0
0,37,34,37,46,50,45,40,39,39,38,...,34,34,35,36,37,36,36,37,38,0
0,25,33,40,42,40,38,36,35,42,46,...,33,38,38,32,31,51,43,41,41,0
0,27,34,36,30,31,35,32,23,20,12,...,24,23,23,27,40,8,27,33,20,0
0,186,192,175,189,202,182,181,188,191,193,...,194,196,185,182,196,193,190,191,194,0
0,34,37,7,25,14,19,22,26,34,13,...,14,15,19,20,17,8,30,21,15,0
0,40,5,5,0,34,8,16,38,22,40,...,35,12,0,57,31,29,6,47,59,0
0,232,251,223,238,237,221,235,232,231,216,...,20,37,35,42,22,56,37,41,13,0
0,232,228,228,229,230,234,212,138,45,45,...,232,235,199,69,26,34,54,49,96,0
0,223,229,220,216,226,222,217,231,238,225,...,229,234,228,222,223,227,227,227,227,0


In [ ]:
## *** Data augmentation *** ##

# Noise:



In [176]:
#### Running models on the unaugmented data
